# 第5回講義 演習

In [1]:
import os
import sys
#sys.path.insert(0, '/root/userspace/public/chap05/materials')
import math

import numpy as np
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

random_seed = 34

np.random.seed(random_seed)
tf.set_random_seed(random_seed)

## 目次

課題. TensorFlowを学ぶ
1. TensorFlow概観
2. TensorFlow基礎
    - 2.1. `Placeholder`・`Variable`
    - 2.2. 行列・テンソル
    - 2.3. 数学
    - 2.4. 制御構文
    - 2.5. 勾配 (微分) の計算
    - 2.6. 変数の値の更新
3. TensorFlow応用
    - 3.1. TensorBoardによるグラフの表示
    - 3.2. グラフの管理と整理
    - 3.3. モデルの保存・読み込み
    - 3.4. メモリの管理
    - 3.5. `tf.data.Dataset`の利用
4. TensorFlowによるニューラルネットワークの実装
    - 4.1. Optimizer
    - 4.2. 正則化 (重み減衰)
    - 4.3. Dropout
    - 4.4. MLP (`tf.data.Dataset`を使用しない版)
    - 4.5. MLP (`tf.data.Dataset`を使用する版)

## 課題. TensorFlowを学ぶ

## 1. TensorFlow概観

`tf`では、基本的に以下の流れで機械学習モデルを構築していきます。

1. `Placeholder`と`Variable`の設定
2. グラフの構築
3. 誤差関数の設定
4. 重みの更新ルールの設定
5. `tf.Session()`を開始して学習
6. 予測

以下にロジスティック回帰 (OR) の例を示します。

In [2]:
# データ (OR)
x_data = np.array([[0, 1], [1, 0], [0, 0], [1, 1]]).astype(np.float32)
t_data = np.array([[1], [1], [0], [1]]).astype(np.float32)

In [3]:
# Step 1. Placeholder・Variableの設定
## Placeholder: データを流し込む変数. データ毎に変わる
x = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='x')
t = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='t')

## Variable: 変数 (重み). データ間で共有される
W = tf.Variable(tf.random_uniform(shape=(2, 1), minval=-0.08, maxval=0.08, dtype=tf.float32), name='W')
b = tf.Variable(tf.zeros(shape=(1), dtype=tf.float32), name='b')

# Step 2. グラフの構築
y = tf.nn.sigmoid(tf.matmul(x, W) + b)

# Step 3. 誤差関数の設定
cost = - tf.reduce_mean(tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y), axis=1))

# Step 4. 重みの更新ルールの設定
gW, gb = tf.gradients(cost, [W, b]) # 勾配の計算
updates = [
    W.assign_sub(0.5 * gW), # 勾配降下法
    b.assign_sub(0.5 * gb)
]
train = tf.group(*updates)

# Step 5. tf.Session()を開始して学習
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 重み (Variable) の初期化
for epoch in range(1000):
    cost_, _ = sess.run([cost, train], feed_dict={x: x_data, t: t_data})
    
    if (epoch + 1) % 100 == 0:
        print('EPOCH: {}, Train Cost, {:.3f}'.format(epoch + 1, cost_))

# Step 6. 予測
print()
y_pred = sess.run(y, feed_dict={x: x_data})
y_pred

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
EPOCH: 100, Train Cost, 0.156
EPOCH: 200, Train Cost, 0.090
EPOCH: 300, Train Cost, 0.062
EPOCH: 400, Train Cost, 0.047
EPOCH: 500, Train Cost, 0.038
EPOCH: 600, Train Cost, 0.032
EPOCH: 700, Train Cost, 0.027
EPOCH: 800, Train Cost, 0.024
EPOCH: 900, Train Cost, 0.021
EPOCH: 1000, Train Cost, 0.019



array([[0.9835751 ],
       [0.9835756 ],
       [0.04120126],
       [0.99998796]], dtype=float32)

## 2. TensorFlow基礎

### 2.1. `Placeholder`・`Variable`

tfには2種類の変数 (のようなもの) があります。それぞれ以下のように使い分けます。

- `tf.placeholder`: データ間で値が共有されない変数 (入力の`x`、正解ラベルの `t` などに使用)
- `tf.Variable` : データ間で値が共有される変数 (重みの `W`、`b` など更新されるものに使用）

#### 2.1.1. `tf.placeholder`

データを流し込む入り口として使います。
- 変数の型 (`tf.int32`、`tf.float32`) を指定する必要があります。
- 実行時にはデータを`feed_dict`で渡す必要があります。

In [4]:
x = tf.placeholder(dtype=tf.float32)

y = x**2

with tf.Session() as sess:
    print(sess.run(y, feed_dict={x: 3}))

9.0


shapeが決まっている場合はそれをリストで指定することもできます。

In [5]:
x = tf.placeholder(dtype=tf.float32, shape=(None, 4))

y = x**2

with tf.Session() as sess:
    print(sess.run(y, feed_dict={x: np.random.randn(8, 4)}))

[[5.94555587e-02 5.58484495e-01 2.43727374e+00 2.15530962e-01]
 [1.23947896e-01 1.64222145e+00 8.36940557e-02 9.60455894e-01]
 [2.28411570e-01 2.03236982e-01 5.66171110e-01 2.60683537e-01]
 [4.98086065e-01 1.80045128e-01 5.38992546e-02 3.29473495e+00]
 [4.38797522e+00 1.07320511e+00 2.05675960e+00 7.74112120e-02]
 [3.54197435e-03 1.71168077e+00 1.65457277e-06 1.59023929e+00]
 [3.47166322e-02 2.04469323e+00 6.25842333e-01 8.86486322e-02]
 [9.25347030e-01 1.14584315e+00 1.45315349e+00 4.14775521e-01]]


#### 2.1.2. `tf.Variable`

値がデータ間で共有されるので、まず初期値を与える必要があります。

- 全ての変数を初期化する場合は`tf.global_variables_initializer()`を使います。
- 個別に変数を初期化する場合は`tf.variables_initializer()`を使い、引数に初期化したい変数をリストで渡します。

In [6]:
w = tf.Variable(1.0, name='w')
b = tf.Variable(0.0)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#     sess.run(tf.variables_initializer([w, b])) # こちらでも可
    
    print(w.eval()) # print(sess.run(w))でも同じです
    print(b.eval())

1.0
0.0


In [ ]:
# w = tf.Variable(0.0)
# b = tf.Variable(1.0)

# with tf.Session() as sess:
#     sess.run(tf.variables_initializer([w]))
#     print(w.eval()) 
#     print(b.eval()) # 初期化していないので、エラーが出ます。

### 2.2. 行列・テンソル

#### 2.2.1. 生成

基本的な行列は、NumPyと同様の関数を使って作ることが可能です。

In [7]:
# スカラー
x_constant = tf.constant(value=9)

# 全要素が1の行列
x_ones = tf.ones((2, 3))

# 全要素が0の行列
x_zeros = tf.zeros((2, 3))

# 指定した値で満たされた行列
x_fill = tf.fill(dims=(2, 3), value=99)

# 単位行列
x_eye = tf.eye(2)

# 渡された変数と同じshapeのtf.ones
x_ones_like = tf.ones_like(tensor=x_eye)

# 渡された変数と同じshapeのtf.zeros
x_zeros_like = tf.zeros_like(tensor=x_eye)

with tf.Session() as sess:
    print('# tf.constant:')
    print(sess.run(x_constant))
    print()
    
    print('# tf.ones:')
    print(sess.run(x_ones))
    print()
    
    print('# tf.zeros:')
    print(sess.run(x_zeros))
    print()
    
    print('# tf.fill:')
    print(sess.run(x_fill))
    print()

    print('# tf.eye:')
    print(sess.run(x_eye))
    print()

    print('# tf.ones_like:')
    print(sess.run(x_ones_like))
    print()

    print('# tf.zeros_like:')
    print(sess.run(x_zeros_like))
    print()

# tf.constant:
9

# tf.ones:
[[1. 1. 1.]
 [1. 1. 1.]]

# tf.zeros:
[[0. 0. 0.]
 [0. 0. 0.]]

# tf.fill:
[[99 99 99]
 [99 99 99]]

# tf.eye:
[[1. 0.]
 [0. 1.]]

# tf.ones_like:
[[1. 1.]
 [1. 1.]]

# tf.zeros_like:
[[0. 0.]
 [0. 0.]]



これらを`Variable`の初期値として利用することも可能です。また代わりにNumPyのarrayを初期値として与えることも可能です。

In [8]:
W = tf.Variable(tf.random_normal((2, 3)), name='w')
b = tf.Variable(np.zeros(3), name='b') # tf.zeros(3)でもOK

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    print(W.eval())
    print(b.eval())

[[ 0.65697956  0.9696641   0.42789015]
 [ 0.8992823  -0.42343983 -0.84637344]]
[0. 0. 0.]


#### 2.2.2. 乱数

一般的な乱数は大体揃っています.

乱数のシードはグローバルに`tf.set_random_seed`で指定するか、個別にseedオプションで指定します。

In [9]:
# seedはグラフレベルでリセットされるので, グラフの初期化も必要. (グラフの初期化については後述)
tf.reset_default_graph()
tf.set_random_seed(34)

# 正規分布
x_randn = tf.random_normal(shape=(2, 3), mean=0.0, stddev=1.0, seed=34) # seedはseedで指定

# 一様分布
x_uniform = tf.random_uniform(shape=(2, 3), minval=0, maxval=1)

# ベルヌーイ分布
x_bernoulli = tf.distributions.Bernoulli(probs=0.5).sample(sample_shape=(10))

with tf.Session() as sess:
    print('# randn:')
    print(sess.run(x_randn))
    print()
    
    print('# uniform:')
    print(sess.run(x_uniform))
    print()
    
    print('# bernoulli:')
    print(sess.run(x_bernoulli))
    print()

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
# randn:
[[ 0.49384186  0.91458374 -0.63111424]
 [-1.3303034  -1.2144839   0.04439415]]

# uniform:
[[0.5487499  0.13401854 0.70727   ]
 [0.04730201 0.74189115 0.40300822]]

# bernoulli:
[1 0 0 0 1 0 1 0 0 0]



#### 2.2.3. shapeの確認

グラフ上の変数のshapeは`tf.shape`で取得することができます。例えば入力データのshapeに合わせて乱数を発生させたいときなどに使います。

In [10]:
x = tf.placeholder(tf.float32, name='x')

x_shape = tf.shape(x)

x_randn = tf.random_normal(shape=x_shape)

x_data = np.empty((2, 3))

with tf.Session() as sess:
    print('# xのshape:')
    print(sess.run(x_shape, feed_dict={x: x_data}))
    print()
    
    print('# normal 1:')
    print(sess.run(x_randn, feed_dict={x: x_data}))
    print()
    
    print('# normal 2:')
    print(sess.run(x_randn, feed_dict={x: x_data[:, :2]}))

# xのshape:
[2 3]

# normal 1:
[[ 0.6309411   0.6158685  -1.4873706 ]
 [-0.13940305 -0.36543897  0.5966492 ]]

# normal 2:
[[ 1.0934397   1.2373759 ]
 [-0.02992922  1.0451084 ]]


#### 2.2.4. 変形

In [11]:
# 全要素が1の行列
x = tf.ones((2, 3))

# 次元の追加: 指定したaxisに新たに次元を追加
x_expand_dims = tf.expand_dims(x, axis=2) # x[:, :, None], x[:, :, tf.newaxis] でも同じ

# 次元の入れ替え
x_transpose = tf.transpose(x, perm=(1, 0))

# 変形
x_reshape = tf.reshape(x, shape=(6, 1))

# 繰り返し
x_tile = tf.tile(input=x, multiples=(2, 1))

# 分割: axisに沿って変数をnum_or_size_splits個に分割
x_split1, x_split2 = tf.split(value=x, num_or_size_splits=2, axis=0)

# 連結
x_concat = tf.concat([x_split1, x_split2], axis=0)

with tf.Session() as sess:
    print('# x:')
    print(sess.run(x))
    print()
    
    print('# tf.expand_dims:')
    print(sess.run(x_expand_dims))
    print()
    
    print('# tf.transpose:')
    print(sess.run(x_transpose))
    print()
    
    print('# tf.reshape:')
    print(sess.run(x_reshape))
    print()
    
    print('# tf.tile:')
    print(sess.run(x_tile))
    print()
    
    print('# tf.split:')
    print(sess.run([x_split1, x_split2]))
    print()
    
    print('# tf.concat')
    print(sess.run(x_concat))
    print()

# x:
[[1. 1. 1.]
 [1. 1. 1.]]

# tf.expand_dims:
[[[1.]
  [1.]
  [1.]]

 [[1.]
  [1.]
  [1.]]]

# tf.transpose:
[[1. 1.]
 [1. 1.]
 [1. 1.]]

# tf.reshape:
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]

# tf.tile:
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

# tf.split:
[array([[1., 1., 1.]], dtype=float32), array([[1., 1., 1.]], dtype=float32)]

# tf.concat
[[1. 1. 1.]
 [1. 1. 1.]]



#### 2.2.5. 行列・テンソル積

`np` の `dot`、`matmul` に対応するものは `tf.matmul` ですが、少し挙動が違うので注意する必要があります。

##### 行列積

`tf.matmul`もしくは`@`を使用します。

In [12]:
a = tf.ones((2, 3))
b = tf.ones((3, 4))

c = tf.matmul(a, b) # a @ b でも可

with tf.Session() as sess:
    print(c.eval())

[[3. 3. 3. 3.]
 [3. 3. 3. 3.]]


ベクトルに対しても `None` などで明示的に行列に変換する必要があります。

In [13]:
a = tf.ones((2,2))
b = tf.ones(2)

# c = tf.matmul(a, b) # エラー
c = tf.matmul(a, b[:, None])

with tf.Session() as sess:
    print(c.eval())

[[2.]
 [2.]]


#### 2.2.6. テンソル積

`np`と同様に`tf`にも`einsum`があります。3階以上のテンソルを含む計算はこれを用いるのがわかりやすくベターです

In [15]:
a = tf.ones((2,3,4))
b = tf.ones((2,3))

c = tf.einsum('ijk,ij->k', a, b)
c_sum = tf.einsum('ijk,ij->', a, b)

with tf.Session() as sess:
    print(c.eval())
    print(c_sum.eval())

[6. 6. 6. 6.]
24.0


### 2.3. 数学

#### 2.3.1. スカラー演算

APIはNumPyと非常に似ています。

In [16]:
np.random.seed(34)

x = tf.placeholder(tf.float32)

x_exp = tf.exp(x)
x_log = tf.log(x)
x_sqrt = tf.sqrt(x)

x_data = np.random.random(2)
print(x_data)
print()
with tf.Session() as sess:
    print('# exp:')
    print(sess.run(x_exp, feed_dict={x: x_data}))
    print()
    
    print('# log:')
    print(sess.run(x_log, feed_dict={x: x_data}))
    print()
    
    print('# sqrt:')
    print(sess.run(x_sqrt, feed_dict={x: x_data}))
    print()

[0.03856168 0.78010046]

# exp:
[1.0393149 2.1816914]

# log:
[-3.2554963  -0.24833256]

# sqrt:
[0.19637127 0.88323295]



ニューラルネットワークなどで使われる活性化関数は`tf.nn`以下にあります。

In [14]:
x = tf.placeholder(tf.float32)

x_sigmoid = tf.nn.sigmoid(x)

x_tanh = tf.nn.tanh(x) # tf.tanhでも可

x_relu = tf.nn.relu(x)

x_elu = tf.nn.elu(x)

x_softplus = tf.nn.softplus(x)

# 正規化したい軸を指定する (ver. 1.4以前はdim、ver. 1.5以降ではaxis)。最後の軸に対して行いたいときは-1
x_softmax = tf.nn.softmax(x, dim=-1)

x_data = np.random.random((2, 3))
print('# x:')
print(x_data)
print()

with tf.Session() as sess:
    print('# sigmoid:')
    print(sess.run(x_sigmoid, feed_dict={x: x_data}))
    print()
    
    print('# tanh:')
    print(sess.run(x_tanh, feed_dict={x: x_data}))
    print()
    
    print('# relu:')
    print(sess.run(x_relu, feed_dict={x: x_data}))
    print()

    print('# elu:')
    print(sess.run(x_elu, feed_dict={x: x_data}))
    print()
    
    print('# softplus:')
    print(sess.run(x_softplus, feed_dict={x: x_data}))
    print()

    print('# softmax:')
    print(sess.run(x_softmax, feed_dict={x: x_data}))
    print()

Instructions for updating:
dim is deprecated, use axis instead
# x:
[[0.80184854 0.51211231 0.07866238]
 [0.03415976 0.56615131 0.14967356]]

# sigmoid:
[[0.6903697  0.6253015  0.51965547]
 [0.5085391  0.63787466 0.5373487 ]]

# tanh:
[[0.665069   0.4715894  0.07850052]
 [0.03414648 0.51252717 0.14856581]]

# relu:
[[0.80184853 0.5121123  0.07866238]
 [0.03415976 0.5661513  0.14967357]]

# elu:
[[0.80184853 0.5121123  0.07866238]
 [0.03415976 0.5661513  0.14967357]]

# softplus:
[[1.1723764 0.9816336 0.7332517]
 [0.7103729 1.0157648 0.7707817]]

# softmax:
[[0.44769472 0.33508205 0.21722321]
 [0.26145372 0.44507766 0.2934687 ]]



#### 2.3.2. 集約演算

引数`axis`で指定した軸に沿って演算を行います。

In [15]:
x = tf.random_normal((2, 3))

x_sum = tf.reduce_sum(x, axis=0)

# 平均, 分散
x_mean, x_var = tf.nn.moments(x, axes=0) # 平均はtf.reduce_meanでも可

# 最大値
x_max = tf.reduce_max(input_tensor=x, axis=0)

# 最小値
x_min = tf.reduce_min(input_tensor=x, axis=0)

# 最大値のindex
x_argmax = tf.argmax(input=x, axis=0)

# 最小値のindex
x_argmin = tf.argmin(input=x, axis=0)

# ノルム
x_norm = tf.norm(tensor=x, ord=2, axis=0)

with tf.Session() as sess:
    x_, x_sum_, x_mean_, x_var_, x_max_, x_min_, x_argmax_, x_argmin_, x_norm_ = sess.run(
        [x, x_sum, x_mean, x_var, x_max, x_min, x_argmax, x_argmin, x_norm])
    
    print('# x:')
    print(x_)
    print()
    
    print('# 合計:')
    print(x_sum_)
    print()
    
    print('# 平均:')
    print(x_mean_)
    print()
    
    print('# 分散:')
    print(x_var_)
    print()

    print('# 最大値:')
    print(x_max_)
    print()
    
    print('# 最小値:')
    print(x_min_)
    print()
    
    print('# 最大値のindex:')
    print(x_argmax_)
    print()
    
    print('# 最小値のindex:')
    print(x_argmin_)
    print()
    
    print('# ノルム:')
    print(x_norm_)
    print()

# x:
[[ 0.7848821  -1.5550356  -0.17887385]
 [-0.14428262 -0.28336897 -0.37591296]]

# 合計:
[ 0.6405995 -1.8384045 -0.5547868]

# 平均:
[ 0.32029974 -0.91920227 -0.2773934 ]

# 分散:
[0.2158368  0.40428397 0.0097061 ]

# 最大値:
[ 0.7848821  -0.28336897 -0.17887385]

# 最小値:
[-0.14428262 -1.5550356  -0.37591296]

# 最大値のindex:
[0 1 0]

# 最小値のindex:
[1 0 1]

# ノルム:
[0.7980335  1.5806434  0.41630086]



### 2.4. 制御構文

#### 2.4.1. 条件

通常のPythonの`if`に対応するものは`tf.cond`です。

In [16]:
x = tf.placeholder(tf.float32, name='x')
y = tf.placeholder(tf.float32, name='y')

absl = tf.cond(
    pred=x > y,
    true_fn=lambda: x - y,
    false_fn=lambda: y - x
)

with tf.Session() as sess:
    print(sess.run(absl, feed_dict={x: 100, y:  50}))
    print(sess.run(absl, feed_dict={x:  50, y: 100}))

50.0
50.0


変数の各要素に対して条件付ける場合は`tf.where`を使います。

In [20]:
x = tf.random_normal((2, 3))

x_where = tf.where(
    condition=x > 0,
    x=tf.ones_like(x),
    y=tf.zeros_like(x)
)

with tf.Session() as sess:
    x_, x_where_ = sess.run([x, x_where])
    
    print('# x:')
    print(x_)
    print()
    
    print('# tf.where(x):')
    print(x_where_)
    print()

# x:
[[-0.3263399  -0.7688404   1.7880154 ]
 [-1.8622565  -0.58020264  0.6365889 ]]

# tf.where(x):
[[0. 0. 1.]
 [0. 0. 1.]]



変数を一定の値でclipしたい場合は`tf.clip_by_value`を使います。例えばlogの中身が0になるのを防ぐときなどに使用します。

In [21]:
x = tf.random_normal((2, 3))

x_log = tf.log(tf.clip_by_value(t=x, clip_value_min=1e-7, clip_value_max=x))

with tf.Session() as sess:
    x_, x_log_ = sess.run([x, x_log])
    
    print('# x:')
    print(x_)
    print()
    
    print('# tf_log(x):')
    print(x_log_)
    print()

# x:
[[ 1.51674     2.0579255   0.44247675]
 [ 1.801229    0.10361186 -0.41155642]]

# tf_log(x):
[[  0.41656327   0.7216984   -0.81536734]
 [  0.5884692   -2.2671034  -16.118095  ]]



`tf.log`などよく使うものは関数化しておくと便利です。

In [22]:
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))

#### 2.4.2. 比較

比較演算子は
- `tf.equal`
- `tf.not_equal`
- `tf.greater`
- `tf.less`

などを使います。

In [23]:
x = tf.placeholder(tf.float32, name='x')
y = tf.placeholder(tf.float32, name='y')

absl = tf.cond(
    pred=tf.greater(x, y),
    true_fn=lambda: x - y,
    false_fn=lambda: y - x
)

with tf.Session() as sess:
    print(sess.run(absl, feed_dict={x: 100, y:  50}))
    print(sess.run(absl, feed_dict={x:  50, y: 100}))

50.0
50.0


他の言語の`for`、`while`に対応するものはそれぞれ`tf.scan`、`tf.while_loop`です。これはRNNの回で詳細に扱います。

### 2.5. 勾配 (微分) の計算

`tf.gradients`をつかうことで微分を計算することができます。

In [24]:
x = tf.placeholder(tf.float32, name='x')
y = x**2

grads = tf.gradients(y, x)

with tf.Session() as sess:
    print(sess.run(grads, feed_dict={x: 1.}))
    print(sess.run(grads, feed_dict={x: 2.}))

[2.0]
[4.0]


第二引数(`xs`)に複数の変数を指定すると、それぞれに対する偏微分をリストで返します。

In [25]:
x1 = tf.placeholder(tf.float32, name='x1')
x2 = tf.placeholder(tf.float32, name='x2')
y = 3*x1**2 + 2*x2**4

grads = tf.gradients(y, [x1, x2])

with tf.Session() as sess:
    print(sess.run(grads, feed_dict={x1: 1, x2: 2}))
    print(sess.run(grads, feed_dict={x1: 3, x2: 4}))

[6.0, 64.0]
[18.0, 512.0]


### 2.6. 変数の値の更新

`Variable`の値を更新するには `assign`、`assign_add`、`assign_sub`を使用します。

In [26]:
a = tf.Variable(0.0, name='w')

increment_a = a.assign_add(1.)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):
        print(sess.run(increment_a))

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0


複数の更新をまとめる場合は `tf.group` を使用します。

In [27]:
a = tf.Variable(0.0, name='a')
b = tf.Variable(10.0, name='b')

increment_a = a.assign_add(1.)
decrement_b = b.assign_sub(1.)

updates = [
    increment_a,
    decrement_b
]

update = tf.group(*updates)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):
        sess.run(update)
        print('a:', a.eval(), end=',  ')
        print('b:', b.eval())

a: 1.0,  b: 9.0
a: 2.0,  b: 8.0
a: 3.0,  b: 7.0
a: 4.0,  b: 6.0
a: 5.0,  b: 5.0
a: 6.0,  b: 4.0
a: 7.0,  b: 3.0
a: 8.0,  b: 2.0
a: 9.0,  b: 1.0
a: 10.0,  b: 0.0


更新に順序をつけたい場合は`tf.control_dependencies`を利用します。

Optimizerを実装するときなどに使用します。

In [28]:
tf.reset_default_graph()

a = tf.Variable(0, name='a')

increment_a = a.assign_add(1)
with tf.control_dependencies([increment_a]):
    square_a = a.assign(a * a)

update = tf.group(*[increment_a, square_a])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5):
        sess.run(update)
        print(sess.run(a))

1
4
25
676
458329


## 3. TensorFlow応用

### 3.1. TensorBoardによるグラフの表示

TensorFlowで計算グラフの構築方法を扱ってきましたが、ここでは構築した計算グラフの可視化をし、視覚的に捉えてみましょう。

計算グラフを表示するには、`tensorboard.py` を読み込む必要があります。

tensorboard.pyをimportしたら、show_graph関数にグラフを渡すことで可視化できます。

可視化結果はインタラクティブな表示になるので、拡大や移動、詳細表示等を試してみましょう。

出典: jupyter上に、tensorBoardのグラフを表示させる
: https://qiita.com/kegamin/items/887c7dfe8bbb76197741 (2018年9月20日参照)

In [17]:
import tensorboard as tb

a = tf.placeholder(tf.float32, name='a')
b = tf.placeholder(tf.float32, name='b')

c = a + b

with tf.Session() as sess:
    print(sess.run([c], feed_dict={a:2, b:3}))

tb.show_graph(sess.graph)    # 単純な足し算のグラフの表示 (がしたいが...)

[5.0]


### 3.2. グラフの管理と整理

#### 3.2.1. デフォルトグラフによる管理

`tf`では特に何も指定しなければ、デフォルトグラフと呼ばれるグラフ上に計算グラフが構築されていきます。
一度計算グラフ上に配置されたグラフは、そのグラフを使うか使わないかにかかわらず、全てリセットされることなく蓄積されていきます。

In [18]:
# 今までの実行によりデフォルトグラフ上に溜まったオペレーション
tf.get_default_graph().get_operations()

[<tf.Operation 'random_normal/shape' type=Const>,
 <tf.Operation 'random_normal/mean' type=Const>,
 <tf.Operation 'random_normal/stddev' type=Const>,
 <tf.Operation 'random_normal/RandomStandardNormal' type=RandomStandardNormal>,
 <tf.Operation 'random_normal/mul' type=Mul>,
 <tf.Operation 'random_normal' type=Add>,
 <tf.Operation 'random_uniform/shape' type=Const>,
 <tf.Operation 'random_uniform/min' type=Const>,
 <tf.Operation 'random_uniform/max' type=Const>,
 <tf.Operation 'random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'random_uniform/sub' type=Sub>,
 <tf.Operation 'random_uniform/mul' type=Mul>,
 <tf.Operation 'random_uniform' type=Add>,
 <tf.Operation 'Bernoulli/probs' type=Const>,
 <tf.Operation 'Bernoulli/logits/Log' type=Log>,
 <tf.Operation 'Bernoulli/logits/mul/x' type=Const>,
 <tf.Operation 'Bernoulli/logits/mul' type=Mul>,
 <tf.Operation 'Bernoulli/logits/Log1p' type=Log1p>,
 <tf.Operation 'Bernoulli/logits/sub' type=Sub>,
 <tf.Operation 'Bernoulli/samp

In [31]:
# 今までの実行によりデフォルトグラフ上に溜まったVariables
tf.global_variables()

[<tf.Variable 'a:0' shape=() dtype=int32_ref>]

そのため、このままでは使わないゴミリソースが蓄積され続けてしまいます。

`tf.reset_default_graph`を毎回新しくグラフを構築する際に呼び出すことにより、これを避けることができます。

In [19]:
# グラフのリセット
tf.reset_default_graph()

# プレースホルダーと変数の宣言
x = tf.placeholder(tf.float32, name='x')
t = tf.placeholder(tf.float32, name='t')
W = tf.Variable(tf.random_uniform((5, 3), -1.0, 1.0), name='W')
b = tf.Variable(tf.zeros((3)), name='b')

# グラフの構築
y = tf.add(tf.matmul(x, W), b, name='y')

# 誤差関数の定義
cost = tf.reduce_mean((y - t)**2, name='cost')

tb.show_graph(tf.Session().graph)

#### 3.2.2. namespaceによる管理

`tf.reset_default_graph`を呼び出すことにより、毎回クリーンな状態でグラフを構築していくことができます。
ただグラフが大規模になってくると、以前変数が多くなりグラフtensorboard上などでノードが見づらくなってしまいます。

tf.name_scope関数を使ってノードをグループに分けることにより、この問題を解消することができます。

まとめられたノードは、カーソルをかざすと出てくる右上のプラスマークをクリックすることで展開することができます。

In [33]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, name='x')
t = tf.placeholder(tf.float32, name='t')

with tf.name_scope('variables'):
    W = tf.Variable(tf.random_uniform((5, 3), -1.0, 1.0), name='W')
    b = tf.Variable(tf.zeros((3)), name='b')

with tf.name_scope('model'):
    y = tf.add(tf.matmul(x, W), b, name='y')

with tf.name_scope('training'):
    cost = tf.reduce_mean(tf.square(y - t), name='cost')

tb.show_graph(tf.Session().graph)

#### 3.2.3. グラフの切り分けによる管理

デフォルトグラフではなく、明示的にグラフオブジェクトを作成し、その上にグラフを構築していくことでグラフ環境を他と分けることも可能です。

これは複数のグラフを構築していきたいときなどに便利です。

In [34]:
tf.reset_default_graph() # グラフのリセット

g0 = tf.get_default_graph() # デフォルトグラフオブジェクトを取得することも可能

g1 = tf.Graph() # グラフオブジェクトの作成1

a = tf.constant(2, name='a0') # これはdefault graphへの配置になるので注意
b = a**a

with g1.as_default(): # デフォルトに設定した上で, グラフを構築・操作
    a = tf.constant(2, name='a')
    b = a**a

g2 = tf.Graph() # グラフオブジェクトの作成2

with g2.as_default(): # デフォルトに設定し, グラフを構築
    a = tf.constant(4, name='a')
    x = tf.constant(3, name='x')
    y = a**x

In [35]:
tb.show_graph(g0)

In [36]:
with tf.Session(graph=g1) as sess:
    print(sess.run(b))
tb.show_graph(g1)

4


In [37]:
with tf.Session(graph=g2) as sess:
    print(sess.run(y))
tb.show_graph(g2)

64


### 3.3. モデルの保存・読み込み

`tf.train.Saver`を使用します。

In [38]:
# データ (OR)
x_data = np.array([[0, 1], [1, 0], [0, 0], [1, 1]])
t_data = np.array([[1], [1], [0], [1]])

In [39]:
tf.reset_default_graph()

x = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='x')
t = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='t')

W = tf.Variable(tf.random_uniform(shape=(2, 1), minval=-0.08, maxval=0.08, dtype=tf.float32), name='W')
b = tf.Variable(tf.zeros(shape=(1), dtype=tf.float32), name='b')

y = tf.nn.sigmoid(tf.matmul(x, W) + b)

cost = - tf.reduce_mean(tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y), axis=1))

gW, gb = tf.gradients(cost, [W, b]) # 勾配の計算
updates = [
    W.assign_sub(0.5 * gW), # 勾配降下法
    b.assign_sub(0.5 * gb)
]
train = tf.group(*updates)

sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 重み (Variable) の初期化
for epoch in range(1000):
    cost_, _ = sess.run([cost, train], feed_dict={x: x_data, t: t_data})
    
    if (epoch + 1) % 100 == 0:
        print('EPOCH: {}, Train Cost, {:.3f}'.format(epoch + 1, cost_))

print()
y_pred = sess.run(y, feed_dict={x: x_data})
y_pred

EPOCH: 100, Train Cost, 0.158
EPOCH: 200, Train Cost, 0.090
EPOCH: 300, Train Cost, 0.062
EPOCH: 400, Train Cost, 0.047
EPOCH: 500, Train Cost, 0.038
EPOCH: 600, Train Cost, 0.032
EPOCH: 700, Train Cost, 0.027
EPOCH: 800, Train Cost, 0.024
EPOCH: 900, Train Cost, 0.021
EPOCH: 1000, Train Cost, 0.019



array([[0.9835565 ],
       [0.9835535 ],
       [0.04125246],
       [0.99998796]], dtype=float32)

保存

In [40]:
saver = tf.train.Saver()
saver.save(sess, '/tmp/model.ckpt')

'/tmp/model.ckpt'

In [41]:
sess.close()

読み込み

In [42]:
tf.reset_default_graph()

x = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='x')
t = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='t')

W = tf.Variable(tf.random_uniform(shape=(2, 1), minval=-0.08, maxval=0.08, dtype=tf.float32), name='W')
b = tf.Variable(tf.zeros(shape=(1), dtype=tf.float32), name='b')

y = tf.nn.sigmoid(tf.matmul(x, W) + b)

saver = tf.train.Saver()

sess = tf.Session()
saver.restore(sess, '/tmp/model.ckpt')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/model.ckpt


予測

In [43]:
y_pred = sess.run(y, feed_dict={x: x_data})
y_pred

array([[0.9835565 ],
       [0.9835535 ],
       [0.04125246],
       [0.99998796]], dtype=float32)

In [44]:
sess.close()

### 3.4. メモリの管理

TensorFlowは、デフォルトでは最初のSessionを開始した時点でシステム上のすべてのGPUのすべてのメモリを専有してしまいます。これは共用のサーバーなどでは特に避けなければいけません。(iLect上では特に気にしなくて大丈夫です。）

ここではいくつかの方法で使用するメモリを制御するいくつかの方法を紹介します。

#### 3.4.1. 使用するGPU(数)を制限する

##### 方法 1.

`tf`から見えるGPUを制限します.

In [45]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

##### 方法 2.

指定したgpu上にグラフを展開していきます.

In [46]:
with tf.device('/gpu:0'):
    x = tf.zeros(4)
    # グラフの構築
    # ...

#### 3.4.2 GPU上のメモリ使用量を制限する

##### 方法 1.

`tf`ではGPUメモリ上に最初にオブジェクトが展開される際、そのオブジェクトのサイズにかかわらずGPU上のすべてのメモリを専有してしまいます。

これをオブジェクトのサイズに応じて段階的にメモリが使用されるように変更します。

In [ ]:
import tensorflow as tf

tf.reset_default_graph()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

x = tf.random_normal((100, 100))

with tf.Session(config=config) as sess:
    pass

# ! nvidia-smi

##### 方法 2.

`tf`が一定割合のメモリのみ使用できるように設定します。

In [ ]:
import tensorflow as tf

tf.reset_default_graph()

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5 # 50%のメモリのみ使用

x = tf.random_normal((100, 100))

with tf.Session(config=config) as sess:
    pass

# ! nvidia-smi

詳細は https://www.tensorflow.org/guide/using_gpu を参照してください。

### 3.5. `tf.data` APIの利用

(以下の説明は`tf.data`の公式ドキュメント https://www.tensorflow.org/guide/datasets を元にしています)

いままでは`feed_dict`に毎回NumPyオブジェクト等を流し込むことにより学習ループを回していました。

もう一つの方法として事前にデータを`tf`のオブジェクト`tf.data.Dataset`に変換し、毎回のNumPyオブジェクトからの変換を省くやり方があります。

入力データが大規模になったり複数のGPUを使う際にはこれにより学習の高速化が期待できます。(参考: Tensorflow Performance Guide: https://www.tensorflow.org/performance/performance_guide)

この場合、グラフの構築の際に

1. NumPyオブジェクトから`Dataset`オブジェクトへの変換
2. Iteratorの設定

を行う必要があります。

#### 3.5.1. `tf.data.Dataset`

まず通常のメモリ上のデータ (ndarray等) から`tf.data.Dataset`に変換します。これには`tf.data.Dataset.from_tensor_slices`などを使用します。

In [47]:
# データ (OR)
x_data = np.array([[0, 1], [1, 0], [0, 0], [1, 1]]).astype(np.float32)
t_data = np.array([[1], [1], [0], [1]]).astype(np.float32)

x_train, t_train = x_data, t_data
x_valid, t_valid = x_data, t_data

In [48]:
tf.reset_default_graph()

# ndarrayからDatasetへ変換
dataset = tf.data.Dataset.from_tensor_slices(x_train)

# 複数のオブジェクトをまとめて変換する場合はタプルで渡す
dataset = tf.data.Dataset.from_tensor_slices((x_train, t_train))

# Dataset同士をまとめる場合はDataset.zipを利用
dataset_x = tf.data.Dataset.from_tensor_slices(x_train)
dataset_t = tf.data.Dataset.from_tensor_slices(t_train)

dataset = tf.data.Dataset.zip((dataset_x, dataset_t))

また`tf.placeholder`を利用して空の`Dataset`を作成することも可能です。ただしこの場合はデータを別の方法で流し込む必要があります。

In [49]:
tf.reset_default_graph()

data = tf.placeholder(tf.float32, shape=(None, 2))
dataset = tf.data.Dataset.from_tensor_slices(data)

# 型情報のみが保存される
print(dataset.output_types)
print(dataset.output_shapes)

<dtype: 'float32'>
(2,)


また、`shuffle`、`batch`、`repeat`、`map`等を利用して前処理部分を書くことができます。

In [50]:
tf.reset_default_graph()

# ndarrayからDatasetへ変換
dataset = tf.data.Dataset.from_tensor_slices(x_train)

# 適当な前処理
dataset = dataset.map(lambda x: x * tf.random_normal([], dtype=tf.float32))

# Dataset内のシャッフル (引数でシャッフル時のバッファ数を指定)
dataset = dataset.shuffle(len(x_data))

# データセットを繰り返す回数 (エポック数) を指定
dataset = dataset.repeat(3)

# バッチサイズを指定. 次元が最初に1つ追加される
print(dataset.output_shapes)
dataset = dataset.batch(4)
print(dataset.output_shapes)

(2,)
(?, 2)


#### 3.5.2. `tf.data.Iterator`

次にIteratorにより、`Dataset`内の要素をどのように走査するかを決めていきます。

現在4つのIterator (One-shot, Initializable, Reinitializable, Feedable) が用意されています。後ろに行くほど柔軟性が高いものになっています。またどのIteratorにおいても走査終了時にはエラー ('tf.errors.OutOfRangeError') が投げられます。

##### One-shot iterator
Datasetを一回だけ走査するiteratorです。

In [ ]:
tf.reset_default_graph()

dataset = tf.data.Dataset.from_tensor_slices(x_data).shuffle(len(x_data))

iterator = dataset.make_one_shot_iterator()

x = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            sess.run(x)
        except tf.errors.OutOfRangeError:
            print('End of dataset')
            break

##### Initializable iterator
one-shot iteratorと違い再利用可能なiteratorです。`iterator.initializer`によって初期化してから使用します。

主に`tf.placeholder`などから作成された空の`Dataset`に対して適用します。

訓練データと検証データを`tf.placeholder`で入れ替えたい場合などに便利です。

In [ ]:
tf.reset_default_graph()

data = tf.placeholder(tf.float32, shape=(4, 2))
dataset = tf.data.Dataset.from_tensor_slices(data)
iterator = dataset.make_initializable_iterator()

x = iterator.get_next()

with tf.Session() as sess:
    # Train
    sess.run(iterator.initializer, feed_dict={data: x_train})
    while True:
        try:
            sess.run(x)
        except tf.errors.OutOfRangeError:
            print('End of train dataset')
            break
    
    # Valid
    sess.run(iterator.initializer, feed_dict={data: x_valid})
    while True:
        try:
            sess.run(x)
        except tf.errors.OutOfRangeError:
            print('End of valid dataset')
            break

##### Reinitializable iterator
型情報のみから作成されるIteratorです。

異なる`Dataset`に渡って使用することができます。

In [ ]:
tf.reset_default_graph()

dataset_train = tf.data.Dataset.from_tensor_slices(x_train)
dataset_train = dataset_train.map(lambda x: x + tf.random_normal([]))

dataset_valid = tf.data.Dataset.from_tensor_slices(x_valid)

# Iteratorの型を指定
iterator = tf.data.Iterator.from_structure(
    output_types=dataset_train.output_types,
    output_shapes=dataset_train.output_shapes
)

x = iterator.get_next()

with tf.Session() as sess:
    # Train
    sess.run(iterator.make_initializer(dataset_train)) # 訓練データでiteratorを初期化
    while True:
        try:
            sess.run(x)
        except tf.errors.OutOfRangeError:
            print('End of train dataset')
            break
    
    # Valid
    sess.run(iterator.make_initializer(dataset_valid)) # 検証データでiteratorを初期化
    while True:
        try:
            sess.run(x)
        except tf.errors.OutOfRangeError:
            print('End of valid dataset')
            break

##### Feedable iterator
どのIteratorを操作するかを柔軟に決めることのできるIteratorです。複数のIteratorの中からどれをiterateするかをfeed_dictにより指定して実行します。

Iteratorを初期化することなく複数のIterator間を行き来できるので、たとえば訓練データの一定iterationごとに検証データを回したいなどの場面で有用です。

In [ ]:
tf.reset_default_graph()

dataset_train = tf.data.Dataset.from_tensor_slices((x_train, t_train))
dataset_train = dataset_train.repeat()

dataset_valid = tf.data.Dataset.from_tensor_slices((x_valid, t_valid))

handle = tf.placeholder(tf.string)
iterator = tf.data.Iterator.from_string_handle(
    string_handle=handle,
    output_types=dataset_train.output_types,
    output_shapes=dataset_train.output_shapes
)

iterator_train = dataset_train.make_one_shot_iterator()
iterator_valid = dataset_valid.make_initializable_iterator()

x, t = iterator.get_next()

with tf.Session() as sess:
    handle_train = sess.run(iterator_train.string_handle())
    handle_valid = sess.run(iterator_valid.string_handle())
        
    # Train
    for i in range(500):
        sess.run(x, feed_dict={handle: handle_train})

        # 100回繰り返すごとにvalidation
        if (i + 1) % 100 == 0:
            print('Iteration: {}'.format(i + 1))
            # Valid
            sess.run(iterator_valid.initializer)
            while True:
                try:
                    sess.run(x, feed_dict={handle: handle_valid})
                except tf.errors.OutOfRangeError:
                    print('End of validation dataset')
                    break
            print()

## 4. TensorFlowによるニューラルネットワークの実装

### 4.1. Optimizer

`tf.gradients`を使うことで様々なOptimizerを実装することができます。

(tfにはbuilt-inで様々なOptimizerが実装されていますが、今回のchap05ではそのような高レベルAPIは使用せず、代わりに各自で実装していただきます。宿題においても同様です。）

いくつかのOptimizerを例として以下に示します。また、個々の学習パラメータ (${\bf W}$や${\bf b}$の各要素)を以下では一般化して$\theta\in\mathbb{R}$で表しています。
また、ステップ$t$における誤差関数$E$に対する学習パラメータ$\theta$の微分を$g_{t}$で表します。

#### SGD

\begin{align*}
    \theta_{t+1} = \theta_{t} - \eta \cdot g_{t}
\end{align*}

In [20]:
def sgd(cost, params, eta=0.01):
    grads = tf.gradients(cost, params)
    updates = []
    for param, grad in zip(params, grads):
        updates.append(param.assign_sub(eta * grad))
    return updates

#### Momentum

\begin{align*}
    v_{t} = \gamma v_{t-1} + \eta \cdot g_{t} \\
    \theta_{t+1} = \theta_{t} - v_{t}
\end{align*}

In [21]:
def momentum(cost, params, eta=0.01, gamma=0.9):
    grads = tf.gradients(cost, params)
    updates = []
    for param, grad in zip(params, grads):
        v = tf.Variable(tf.zeros_like(param, dtype=tf.float32), name='v')
        updates.append(v.assign(gamma * v + eta * grad))
        with tf.control_dependencies(updates):
            updates.append(param.assign_sub(v))
    return updates

#### Adagrad

\begin{align*}
    G_{t} = G_{t-1} + g^{2}_t \\
    \theta_{t+1} = \theta_{t} - \frac{\eta}{\sqrt{G_{t} + \epsilon}} \cdot g_{t}
\end{align*}

In [22]:
def adagrad(cost, params, eta=0.01, eps=1e-7):
    grads = tf.gradients(cost, params)
    updates = []
    for param, grad in zip(params, grads):
        G = tf.Variable(tf.zeros_like(param, dtype=tf.float32), name="G")
        updates.append(G.assign_add(grad ** 2))
        with tf.control_dependencies(updates):
            updates.append(param.assign_sub(eta * grad / tf.sqrt(G + eps)))
    return updates

#### RMSProp

\begin{align*}
    \tilde{G}_{t} = \rho \cdot \tilde{G}_{t-1} + (1 - \rho) \cdot g^{2}_{t} \\
    \theta_{t+1} = \theta_{t} - \frac{\eta}{\sqrt{\tilde{G}_{t} + \epsilon}} \cdot g_{t}
\end{align*}

In [23]:
def rmsprop(cost, params, eta=0.001, rho=0.9, eps=1e-7):
    grads = tf.gradients(cost, params)
    updates = []
    for param, grad in zip(params, grads):
        G = tf.Variable(tf.zeros_like(param, dtype=tf.float32), name='G')
        updates.append(G.assign(rho * G + (1 - rho) * grad**2))
        with tf.control_dependencies(updates):
            updates.append(param.assign_sub(eta / tf.sqrt(G + eps) * grad))

    return updates

Adadelta、Adam等も同じ要領で実装できるので、余裕のある人は実装してみましょう。

### 4.2. Dropout

Dropoutを適用するには`tf.nn.dropout`を使います。

引数の`keep_prob`でユニットをキープする確率を設定します。また、関数の出力は$\frac{1}{\text{keep_prob}}$倍されます。

In [24]:
tf.reset_default_graph()
tf.set_random_seed(34)

x = tf.placeholder(tf.float32)

x_dropped = tf.nn.dropout(x, keep_prob=0.5)

with tf.Session() as sess:
    print(sess.run(x_dropped, feed_dict={x: np.ones(10)}))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[0. 2. 2. 2. 2. 2. 2. 0. 2. 0.]


### 4.3. 正則化 (重み減衰)

#### L1正則化
\begin{align*}
    \tilde{E} = E + \lambda \sum_{\theta} |\theta|
\end{align*}

In [25]:
def compute_l1_reg(params):
    l1_reg = 0
    for param in params:
        l1_reg += tf.reduce_sum(tf.abs(param))
    return l1_reg

#### L2正則化
\begin{align*}
    \tilde{E} = E + \lambda \sum_{\theta} \theta^2
\end{align*}

In [26]:
def compute_l2_reg(params):
    l2_reg = 0
    for param in params:
        l2_reg += tf.reduce_sum(tf.square(param))
    return l2_reg

### 4.4. MLPの実装 (`tf.data.Dataset`を使用しない版)

TensorFlowを使ってMLPを実装してみましょう。

データセットにはMNISTを使用します。

In [35]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original', data_home="../../")

x_mnist = mnist.data.astype('float32') / 255.
t_mnist = np.eye(10)[mnist.target.astype('int32')]

x_train_mnist, x_test_mnist, t_train_mnist, t_test_mnist = train_test_split(x_mnist, t_mnist, test_size=10000)
x_train_mnist, x_valid_mnist, t_train_mnist, t_valid_mnist = train_test_split(x_train_mnist, t_train_mnist, test_size=10000)

/Users/takuma.s/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
/Users/takuma.s/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [37]:
#fashion mnist
from sklearn.model_selection import train_test_split
x_train = np.load('../data/x_train.npy')
t_train = np.load('../data/y_train.npy')
x_train = x_train.reshape(-1, 784).astype('float32') / 255
t_train = np.eye(10)[t_train.astype('int32')]
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=1000)

全結合層とDropout層からなるMLPを実装します。

順伝播の式を示します。
$$
\begin{align*}
    {\bf u}^{(1)} &= {\bf W}^{(1)\mathrm{T}} {\bf x} + {\bf b}^{(1)} \\
    {\bf h}^{(1)} &= \mathrm{ReLU}({\bf u}^{(1)}) \\
    {\bf \tilde{h}}^{(1)} &= \mathrm{Dropout}({\bf h}^{(1)}) \\
    {\bf u}^{(2)} &= {\bf W}^{(2)\mathrm{T}} {\bf \tilde{h}}^{(1)} + {\bf b}^{(2)} \\
    {\bf h}^{(2)} &= \mathrm{ReLU}({\bf u}^{(2)}) \\
    {\bf \tilde{h}}^{(2)} &= \mathrm{Dropout}({\bf h}^{(2)}) \\
    {\bf u}^{(3)} &= {\bf W}^{(3)\mathrm{T}} {\bf \tilde{h}}^{(2)} + {\bf b}^{(3)} \\
    {\bf y} &= \mathrm{softmax} ({\bf u}^{(3)})
\end{align*}
$$

全結合層とDropout層を定義します。

Dropout層では、訓練時と検証時のDropoutを適用の有無を`tf.cond`により制御します。

In [38]:
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        self.W = tf.Variable(tf.random_uniform(shape=(in_dim, out_dim), minval=-0.08, maxval=0.08), name='W')
        self.b = tf.Variable(tf.zeros(out_dim), name='b')
        self.function = function
        
        self.params = [self.W, self.b]
    
    def __call__(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)

In [39]:
class Dropout:
    def __init__(self, dropout_keep_prob=1.0):
        self.dropout_keep_prob = dropout_keep_prob
        self.params = []
    
    def __call__(self, x):
        # 訓練時のみdropoutを適用
        return tf.cond(
            pred=is_training,
            true_fn=lambda: tf.nn.dropout(x, keep_prob=self.dropout_keep_prob),
            false_fn=lambda: x
        )

誤差関数は通常の他クラス交差エントロピーにL2正則化の項を追加したものを使用します。
$$
    E = -\frac{1}{N}\sum^N_{i=1} \sum^K_{k=1} {\bf t}_{i, k} \log {\bf y}_{i, k} + \lambda\sum_{\theta}\theta^2
$$

In [40]:
# tf.log(0)によるnanを防ぐ
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))

In [41]:
eta = 0.01 # 学習率
dropout_keep_prob = 0.5 # Dropout率
lmd = 0.001 # L2正則化項の係数
batch_size = 32 # バッチサイズ
n_epochs = 10 # epoch数

In [42]:
tf.reset_default_graph() # グラフのリセット

x = tf.placeholder(tf.float32, (None, 784)) # 入力データ
t = tf.placeholder(tf.float32, (None, 10)) # 教師データ
is_training = tf.placeholder(tf.bool) # 訓練時orテスト時

layers = [
    Dense(784, 200, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(200, 200, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(200, 10, tf.nn.softmax)
]

def get_params(layers):
    params_all = []
    for layer in layers:
        params = layer.params
        params_all.extend(params)
    return params_all

def f_props(layers, h):
    for layer in layers:
        h = layer(h)
    return h

y = f_props(layers, x)
params_all = get_params(layers)
l2_reg = compute_l2_reg(params_all)

cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1)) + lmd * l2_reg

updates = sgd(cost, params_all, eta)
train = tf.group(*updates)

n_batches = math.ceil(len(x_train) / batch_size)

学習

In [43]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        x_train, t_train = shuffle(x_train, t_train)
        for i in range(n_batches):
            start = i * batch_size
            end = start + batch_size
            sess.run(train, feed_dict={x: x_train[start:end], t: t_train[start:end], is_training: True})
        y_pred, cost_valid_ = sess.run([y, cost], feed_dict={x: x_valid, t: t_valid, is_training: False})
        print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
            epoch + 1,
            cost_valid_,
            accuracy_score(t_valid.argmax(axis=1), y_pred.argmax(axis=1))
        ))

EPOCH: 1, Valid Cost: 1.148, Valid Accuracy: 0.713
EPOCH: 2, Valid Cost: 1.004, Valid Accuracy: 0.781
EPOCH: 3, Valid Cost: 0.916, Valid Accuracy: 0.813
EPOCH: 4, Valid Cost: 0.861, Valid Accuracy: 0.829
EPOCH: 5, Valid Cost: 0.813, Valid Accuracy: 0.844
EPOCH: 6, Valid Cost: 0.784, Valid Accuracy: 0.840
EPOCH: 7, Valid Cost: 0.753, Valid Accuracy: 0.843
EPOCH: 8, Valid Cost: 0.728, Valid Accuracy: 0.857
EPOCH: 9, Valid Cost: 0.699, Valid Accuracy: 0.856
EPOCH: 10, Valid Cost: 0.678, Valid Accuracy: 0.860


### 4.5. `tf.data.Dataset`を用いた実装

参考として`tf.data.Dataset`を使用した実装を以下に示します。

In [ ]:
tf.reset_default_graph()

dataset_train = tf.data.Dataset.from_tensor_slices((x_train, t_train))
dataset_train = dataset_train.repeat()
dataset_train = dataset_train.batch(batch_size)
flag_true = tf.data.Dataset.from_tensor_slices(tf.ones(len(x_train), dtype=tf.bool))
flag_true = flag_true.repeat()
dataset_train = tf.data.Dataset.zip((dataset_train, flag_true))

dataset_valid = tf.data.Dataset.from_tensor_slices((x_valid, t_valid))
dataset_valid = dataset_valid.batch(batch_size)
flag_false = tf.data.Dataset.from_tensor_slices(tf.zeros(len(x_valid), dtype=tf.bool))
dataset_valid = tf.data.Dataset.zip((dataset_valid, flag_false))

handle = tf.placeholder(tf.string)
iterator = tf.data.Iterator.from_string_handle(
    string_handle=handle,
    output_types=dataset_train.output_types,
    output_shapes=dataset_train.output_shapes
)

iterator_train = dataset_train.make_one_shot_iterator()
iterator_valid = dataset_valid.make_initializable_iterator()

(x, t), is_training = iterator.get_next()

layers = [
    Dense(784, 200, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(200, 200, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(200, 10, tf.nn.softmax)
]

def get_params(layers):
    params_all = []
    for layer in layers:
        params = layer.params
        params_all += params
    return params_all

def f_props(layers, h):
    for layer in layers:
        h = layer(h)
    return h

y = f_props(layers, x)
params_all = get_params(layers)
l2_reg = compute_l2_reg(params_all)

cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1)) + lmd * l2_reg

updates = sgd(cost, params_all, eta)
train = tf.group(*updates)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    handle_train = sess.run(iterator_train.string_handle())
    handle_valid = sess.run(iterator_valid.string_handle())
    
    for i in range(10000):
        # Train
        sess.run(train, feed_dict={handle: handle_train})
        
        # Valid
        if (i + 1) % 2000 == 0:
            sess.run(iterator_valid.initializer)
            costs_valid = []
            y_preds = []
            while True:
                try:
                    cost_, y_pred = sess.run([cost, y], feed_dict={handle: handle_valid})
                    costs_valid.append(cost_)
                    y_preds.extend(y_pred.argmax(axis=1))
                except tf.errors.OutOfRangeError:
                    break
            print('Iteration: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
                i + 1,
                np.mean(costs_valid),
                accuracy_score(t_valid.argmax(axis=1), y_preds)
            ))